In [ ]:
# Import modules
%pylab inline
rc('animation', html='html5')
import mizer

In [ ]:
# mizer parameters
parameters = {
  'w_min': 1e-3,
  'w_inf': 1e6,
  'nclass': 100,
  'T_dependence': 1,
  'T_ref': 13.,
  'E_a': 0.63,
  'beta': 100,
  'sigma': 2.20,   # ln(10.)
  'gamma': 64.,
  'q': 0.82,
  'alpha': 0.2,
  'z0_type': 1,
  'z0pre': 0.1,
  'z0exp': -0.25,
  'w_s': 1000.,
  'z_s': 0.3,
  'ks': 0.,
  'SRR': 0,
  'recruitment': 10000,
  'h': 1e9,
  'w_minF': 1.25, # Blanchard et al 2012
  'F': 1.6
}

# Function for converting from Equivalent Spherical Diameter (micrometer) to wet mass in g
def esd2mass(d): # d: equivalent spherical diameter in micrometer
    V = 4./3.*pi*(numpy.array(d)/2e6)**3  # V: volume in m3
    return V*1e6  # mass in g approximately equals volume in m3 multiplied by 1e6 (assumes density of 1000 kg/m3)

# prey (currently from GOTM-ERSEM simulation)
path = r'..\..\ssb-ersem-git\testcases\ersem_gotm\L4\fabm_ref.nc'
scale_factor = 0.01 # 10 g wet mass/g carbon * 0.001 g C/mg C
prey = (
   mizer.Prey('diatoms', esd2mass((20,200)), mizer.datasources.TimeSeries(path, 'P1_c', z=-1, scale_factor=scale_factor)),
   mizer.Prey('nanophy', esd2mass((2,20)), mizer.datasources.TimeSeries(path, 'P2_c', z=-1, scale_factor=scale_factor)),
   mizer.Prey('picophy', esd2mass((.2,2)), mizer.datasources.TimeSeries(path, 'P3_c', z=-1, scale_factor=scale_factor)),
   #mizer.Prey('microphy', esd2mass((20,200)), mizer.datasources.TimeSeries(path, 'P4_c', z=-1, scale_factor=scale_factor)),
   mizer.Prey('microzoo', esd2mass((20,200)), mizer.datasources.TimeSeries(path, 'Z5_c', z=-1, scale_factor=scale_factor)),
   mizer.Prey('nanoflag', esd2mass((2,20)), mizer.datasources.TimeSeries(path, 'Z6_c', z=-1, scale_factor=scale_factor)),
   #mesozoo = mizer.Prey('mesozoo', 1e-4, mizer.datasources.TimeSeries(path, 'Z4_c', z=-1, scale_factor=scale_factor)),
)
prey_collection = mizer.PreyCollection(*prey)
prey_collection = mizer.GriddedPreyCollection(prey_collection)

# environment
#temp = mizer.datasources.TimeSeries(path, 'temp', z=-1)
temp = 15.

# create mizer model
m = mizer.Mizer(prey=prey_collection, parameters=parameters, temperature=temp)

In [ ]:
# Time-integrate
times = numpy.arange(datestr2num('2002-01-01'), datestr2num('2010-01-01'))
result = m.run(times, spinup=50, verbose=True, save_spinup=False)

In [ ]:
from ipywidgets import widgets
def show_spectrum(day):
    result.plot_spectrum(day, normalization=0, global_range=True)
slider = widgets.interact(show_spectrum, day=(0, len(times)-1))

In [ ]:
result.plot_biomass_timeseries()
result.plot_timeseries('landings')

In [ ]:
anim = result.animate_spectrum(normalization=0)

In [ ]:
anim